In [9]:
import requests
import json
import pandas as pd
import io
from google.cloud import storage
from google.oauth2 import service_account
from datetime import datetime

In [17]:
import requests
import json
import pandas as pd
import io
from google.cloud import storage
from google.oauth2 import service_account
from datetime import datetime


def get_data_larkbase(token):

    #lay tenant_access_token
    url = "https://open.larksuite.com/open-apis/auth/v3/tenant_access_token/internal"
    payload = json.dumps({
        "app_id": token['app_id'],
        "app_secret": token['app_secret']
    })

    headers = {
    'Content-Type': 'application/json'
    }

    response = requests.request('POST',url, headers=headers, data=payload)

    if response.status_code == 200:
        info = json.loads(response.text)
        tenant_access_token ='Bearer ' + info['tenant_access_token']

    else:
        print(f"Error: {response.status_code} - {response.text}")


    #lay data tu bang Lark Attendance Result can tenant_access_token, app_token, table_id
    app_token = token['app_token']
    table_id = token['table_id']
    page_token = ''
    payload = ''

    headers = {
    'Authorization': tenant_access_token
    }

    records = []

    url = f"https://open.larksuite.com/open-apis/bitable/v1/apps/{app_token}/tables/{table_id}/records?page_size=500&page_token={page_token}"

    response = requests.request("GET", url, headers=headers, data=payload)

    data = json.loads(response.text)
    records.extend(data['data']['items'])

    #lay trang thai has_more
    has_more = data['data']['has_more']


    #1 api lay toi da 500 record, phai loop qua tung trang de lay het data (neu co thi has_more = True)
    while has_more == True:
        page_token = data['data']['page_token']
        url = f"https://open.larksuite.com/open-apis/bitable/v1/apps/{app_token}/tables/{table_id}/records?page_size=500&page_token={page_token}"

        response = requests.request("GET", url, headers=headers, data=payload)

        data = json.loads(response.text)
        records.extend(data['data']['items'])

        has_more = data['data']['has_more']

    result = [record['fields'] for record in records]

    return result


def lark_to_gcs(file_load,df):

    # Sử dụng io.StringIO để ghi DataFrame vào bộ nhớ dưới dạng CSV
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False, float_format='%.0f')

    # Khởi tạo client Google Cloud Storage
    credentials = service_account.Credentials.from_service_account_file('/home/admin1/Documents/App/airflow/dw-demo-435209-d671317c69dd.json')
    client = storage.Client(credentials=credentials)


    # Lấy bucket từ tên bucket đã tạo trên GCS
    bucket = client.get_bucket(file_load['bucket'])

    # Tạo blob (file) để upload
    blob = bucket.blob(file_load['path'])

    # Upload CSV lên blob
    blob.upload_from_string(csv_buffer.getvalue(), content_type='text/csv')




In [27]:
#Attendance_result table
#get data_from_lark
token_employee_table = {
    "app_id": "cli_a66321e17db81010",
    "app_secret": "38ub3IgwcaZB6wpLNVzw4gkYb0Z72Qe3",
    'app_token': 'U0PMbECFHat1KDsd3Bol6qRKg5g',
    'table_id': 'tblzPogj1pOD3LZN'
}

data_attendance_result = get_data_larkbase(token_employee_table)

df_attendance_result = pd.DataFrame({
    'Result id': [record.get('Result id', '') for record in data_attendance_result],
    'User id': [record.get('User id', '') for record in data_attendance_result],
    'Date': [record.get('Date', '') for record in data_attendance_result],
    'Days_week': [record.get('Days_week','') for record in data_attendance_result],
    'Shift name': [record.get('Shift name', '') for record in data_attendance_result],
    'Check in shift time': [record.get('Check in shift time','') for record in data_attendance_result],
    'Check out shift time' : [record.get('Check out shift time','') for record in data_attendance_result]
})

#load data from to gcs
file_load ={
    'bucket': 'attendance_data2024',
    'path': 'attendance_result.csv'
}

load_data = lark_to_gcs(file_load,df_attendance_result)


In [29]:
df_attendance_result

,Result id,User id,Date,Days_week,Shift name,Check in shift time,Check out shift time
0,7376486864720511008,c576b314,1717693200000,6,CN,1717723800000,1717756200000
1,7376486864754081824,c576b314,1717779600000,1,CN,1717794000000,1717880400000
2,7377111916893716511,c576b314,1717866000000,2,CN,1717880400000,1717966800000
3,7377482814620450848,f399c46c,1717952400000,2,8h30-17h30,1717983000000,1718015400000
4,7377285920480903200,42a19972,1717952400000,2,8h30-17h30,1717983000000,1718015400000
...,...,...,...,...,...,...,...
756,7414591757779730463,844e1491,1726592400000,4,8h30-17h30,1726623000000,1726655400000
757,7414591757691650079,f399c46c,1726592400000,4,8h30-17h30,1726623000000,1726655400000
758,7414591757616152608,f4b97g67,1726592400000,4,8h30-17h30,1726623000000,1726655400000
759,7415657894852444191,bed7bgf6,1726592400000,4,8h30-17h30,1726623000000,1726655400000
